# Analyse Chronologique et Sentiment des Actualités d'Entreprise

## Importation des Bibliothèques Nécessaires

In [1]:

import pandas as pd
from textblob import TextBlob
import numpy as np
import json

## Chargement et Organisation des Données

In [2]:
data_google_search = pd.read_csv('GoogleSearchResult-2024-03-03.csv')
company = pd.read_csv('company-2024-03-03.csv')
siret_google_search=pd.merge(company,data_google_search, on='id', how='inner')

In [3]:

# Suppression des colonnes
siret_google_search = siret_google_search.drop(['id','brand_name','address','city','postal_code','country','description','phone','logo'], axis=1)

# Création d'un index à partir de la colonne Siren
siret_google_search = siret_google_search.set_index('siren')
siret_google_search['siren'] = siret_google_search.index

In [4]:

siret_google_search.to_csv('siret_google_search.csv', index='siren')
siret_google_search.info()

<class 'pandas.core.frame.DataFrame'>
Index: 272 entries, 847593225 to 348991555
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              272 non-null    object 
 1   website           1 non-null      object 
 2   siret             272 non-null    object 
 3   ceo_name          269 non-null    object 
 4   ape               272 non-null    object 
 5   founded_date      272 non-null    object 
 6   turnover_2022     174 non-null    float64
 7   gouv_searched     271 non-null    float64
 8   gouv_street       176 non-null    object 
 9   gouv_city         176 non-null    object 
 10  gouv_postal_code  176 non-null    float64
 11  gouv_country      0 non-null      float64
 12  gouv_lat          176 non-null    float64
 13  gouv_lng          176 non-null    float64
 14  search            272 non-null    int64  
 15  company           272 non-null    int64  
 16  google_results    272 non-null    o

In [5]:
siret_google_search.head(5)

,name,website,siret,ceo_name,ape,founded_date,turnover_2022,gouv_searched,gouv_street,gouv_city,gouv_postal_code,gouv_country,gouv_lat,gouv_lng,search,company,google_results,scraped_date,scraped,siren
siren,,,,,,,,,,,,,,,,,,,,
847593225,AAA COIFFURE,NaN,84759322500018,Salah HARROU,9602A,2019-01-15,33300.0,1.0,Rue Paganini,Nice,6000.0,NaN,43.702199,7.263926,116,457,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:41:02,1.0,847593225
331057406,ACA NEXIA,NaN,33105740600067,Olivier LELONG,6920Z,1984-09-01,18602980.0,1.0,Rue Henri Rochefort,Paris,75017.0,NaN,48.882511,2.308276,116,330,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:40:38,1.0,331057406
845298249,AEV AUTO ECOLE VEILLON,NaN,84529824900028,Emna KHALFAOUI,8553Z,2019-01-09,244188.0,1.0,Avenue Villermont,Nice,6000.0,NaN,43.709300,7.264297,116,176,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:40:56,1.0,845298249
316473305,AGRI-POLE,NaN,31647330500035,Robert BARTHES,4661Z,1979-01-01,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,149,280,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:42:26,1.0,316473305
349638395,AIR CORSICA,NaN,34963839500021,Luc BERENI,5110Z,1990-01-01,130850508.0,1.0,Campo Dell Oro,Ajaccio,20090.0,NaN,41.929606,8.804378,116,524,"{""top_ads"": [], ""meta_data"": {""url"": ""https://...",2023-09-20 10:39:46,1.0,349638395


In [7]:

# Fonction pour extraire les données nécessaires de google_search

# Fonction ajustée pour prendre en compte le siren extérieur
def extraire_donnees(json_str, siren):
    try:
        data = json.loads(json_str)
        results = []
        
        # Itération sur les résultats organiques
        for result in data.get('organic_results', []):  # Utilisez .get pour éviter les erreurs si la clé n'existe pas
            # Initialisation du dictionnaire de résultats
            res_dict = {
                'siren': siren,
                'url': result.get('url', ''),
                'title': result.get('title', ''),
                'domain': result.get('domain', ''),
                'description': result.get('description', ''),
                'displayed_url': result.get('displayed_url', '')
            }
            
            # Traitement des sitelinks
            if isinstance(result.get('sitelinks'), dict):
                res_dict['sitelinks'] = result.get('sitelinks', {}).get('expanded', [])
            else:
                res_dict['sitelinks'] = []  # Assurez-vous que 'sitelinks' est toujours une liste
            
            # Ajout du dictionnaire de résultats à la liste des résultats
            results.append(res_dict)
        
        return results
    except Exception as e:
        print("Erreur lors de l'extraction:", e, "siren", siren)  # Pour déboguer
        return []

resultats_extraits = []

# Supposons que 'siret_google_search' est votre DataFrame
# Itérer sur chaque ligne du DataFrame
for idx, row in siret_google_search.iterrows():
    # Extraire les données organiques et le siren pour chaque ligne
    resultats_temp = extraire_donnees(row['google_results'], row['siren'])
    # Ajouter les résultats à la liste principale
    resultats_extraits.extend(resultats_temp)

# Convertir la liste des résultats extraits en un nouveau DataFrame
detail_google_search = pd.DataFrame(resultats_extraits)

In [9]:
detail_google_search.head(5)

,siren,url,title,domain,description,displayed_url,sitelinks
0,847593225,https://clinique-saint-martin.ramsaysante.fr/,Clinique Saint-Martin: Accueil,clinique-saint-martin.ramsaysante.fr,Actualités Santé Mentale. Toutes les actualité...,https://clinique-saint-martin.ramsaysante.fr,[{'link': '/search?num=100&sca_esv=566872717&g...
1,847593225,https://www.elsan.care/fr/nos-actualites?layou...,Actualités,www.elsan.care,Toutes les actualités · Saint-Martin se mobili...,https://www.elsan.care › nos-actualites,[]
2,847593225,https://www.elsan.care/fr/hopital-prive-saint-...,Accueil - Hôpital Privé Saint-Martin,www.elsan.care,L'Hôpital Privé Saint Martin est un établissem...,https://www.elsan.care › hopital-pri...,[{'link': '/search?num=100&sca_esv=566872717&g...
3,847593225,https://www.has-sante.fr/jcms/328_FicheEtablis...,CLINIQUE SAINT MARTIN SUD MARSEILLE,www.has-sante.fr,"Jun 23, 2022 — CLINIQUE SAINT MARTIN SUD MARSE...",https://www.has-sante.fr › jcms › cli...,[]
4,847593225,https://www.iml.care/lien-centre+de+reeducatio...,centre de rééducation marseille Saint Martin s...,www.iml.care,Actualités · Faire un don · Guide local · Cont...,https://www.iml.care › lien-centre+d...,[]


In [10]:
detail_google_search.to_csv('detail_google_search.csv', index='siren')

In [83]:
# Assurez-vous que la colonne de date est au format datetime
data['date'] = pd.to_datetime(data['date'])
data.sort_values('date', inplace=True)

KeyError: 'date'

## Analyse de Sentiment

In [ ]:
def analyze_sentiment(text):
    analysis = TextBlob(text)
    if analysis.sentiment.polarity > 0:
        return 'Positif'
    elif analysis.sentiment.polarity == 0:
        return 'Neutre'
    else:
        return 'Négatif'

data['sentiment'] = data['texte'].apply(analyze_sentiment)

## Classification des Messages

In [ ]:
def classify_message(text):
    # Adaptez ces conditions selon le contenu typique de vos messages
    if 'risque' in text.lower():
        return 'Risque'
    elif 'opportunité' in text.lower() or 'chance' in text.lower():
        return 'Opportunité'
    else:
        return 'Indéterminé'

data['classification'] = data['texte'].apply(classify_message)

## Affichage des Résultats

In [ ]:
data[['date', 'texte', 'sentiment', 'classification']]